In [1]:
from utils import get_klines, build_features_by_data, y_to_bins
from binance.client import Client
import pandas as pd
import numpy as np

from models.random_forest import RandomForest

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
# Hyper parameters:
CUTOFF = 0.15  # in percents. The minimal value of ascending
N_ESTIMATORS = 100


In [2]:
s_date = '01 Jan, 2019'
e_date = '03 Jan 2019'
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_12HOUR


In [8]:

kl_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
# klines = get_klines(s_date, e_date, symbol, interval)
# klines.to_csv('klines/30M/' + kl_f_name, index=False)
klines = pd.read_csv('klines/30M/' + kl_f_name)

In [9]:
klines.head()

,open,high,low,close,volume,Number of trades
0,3701.23,3713.00,3689.88,3700.31,686.367420,5534.0
1,3700.20,3702.73,3684.22,3689.69,613.539115,5086.0
2,3689.67,3695.95,3675.04,3690.00,895.302181,6391.0
3,3690.00,3699.77,3685.78,3693.13,796.714818,5709.0
4,3692.32,3720.00,3685.94,3692.71,1317.452909,7908.0


In [20]:
features_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
# features = build_features_by_data(klines)
# features.to_csv('features/30M/' + features_f_name, index=False)
features = pd.read_csv('features/30M/' + features_f_name)
features['TO_PREDICT'] = features['close'].shift(-1)
features['TO_PREDICT'] = features['TO_PREDICT']/features['close']


In [21]:

features = y_to_bins(features, 'TO_PREDICT', CUTOFF)

In [22]:
l = features.shape[0]
X_train = np.array(features.iloc[:int(0.8*l)].drop('y_bins', axis=1))
X_valid = np.array(features.iloc[int(0.8*l):].drop('y_bins', axis=1))
y_train = np.array(features['y_bins'].iloc[:int(0.8*l)])
y_valid = np.array(features['y_bins'].iloc[int(0.8*l):])
df_valid = features.iloc[int(0.8*l):]

In [23]:
model = RandomForest(n_estimators=N_ESTIMATORS)

In [24]:
model.fit(
    X=X_train,
    y=y_train)

In [25]:
df_valid['predictions'] = model.predict(df_valid.drop('y_bins', axis=1))
pred_file_name = 'try.csv'
df_valid.to_csv('predictions/'+pred_file_name)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
